In [ ]:
from queue import Queue
from threading import Thread
from typing import Dict, List
from PIL import Image
import cv2
import numpy as np

import face_tool
from face_model import FaceModel
from fps_metric import FPS
from scrfd import SCRFD
from utils import check_in_out, get_data, maybe_create_folder, open_cam_usb
# import mediapipe as mp
import time
from web_cam import WebcamVideoStream
import glob

In [2]:
!pip3 install face_recognition

In [1]:
import dlib

ImportError: dlopen(/Users/dominhtuan/Downloads/AnNinhSoiChieu/env/lib/python3.12/site-packages/_dlib_pybind11.cpython-312-darwin.so, 0x0002): tried: '/Users/dominhtuan/Downloads/AnNinhSoiChieu/env/lib/python3.12/site-packages/_dlib_pybind11.cpython-312-darwin.so' (mach-o file, but is an incompatible architecture (have (x86_64), need (arm64e)))

In [3]:
import face_recognition

AttributeError: module 'dlib' has no attribute 'get_frontal_face_detector'

In [8]:
mpFaceDetection = mp.solutions.face_detection 
mpDraw = mp.solutions.drawing_utils
faceDetection = mpFaceDetection.FaceDetection()

I0000 00:00:1718957704.329149 2297169 gl_context.cc:357] GL version: 2.1 (2.1 Metal - 76.3), renderer: Apple M1


In [9]:
face_detector = SCRFD(model_file='/Users/dominhtuan/Downloads/AnNinhSoiChieu/scrfd_10g_bnkps.onnx')
face_model = FaceModel(onnx_model_path='/Users/dominhtuan/Downloads/AnNinhSoiChieu/webface_r50.onnx')

width = 1280
height = 720
WINDOW_NAME = 'CameraCheckIn'

W0000 00:00:1718957704.331483 2307995 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.


In [10]:
def start_app(video_capture):
    pTime = 0
    frame_index = 0
    face_step = 2
    detect_step = 4
    reset_step = 120

    fps = FPS().start()
    while True:
        
        
        
        # frame_index +=1
        _, frame = video_capture.read()
        flag = False
        frame = cv2.flip(frame, 1)
        face_list = []
        name ="Unknown"
        if frame_index % detect_step == 0:
            rs = faceDetection.process(frame)
            # face_boxes, kpss = face_detector.detect(frame)
            # if face_boxes is not None:
            #     face_boxes = face_boxes.astype(np.int32)
            #     face_boxes = face_boxes[:, :4]
            #     face_boxes = np.maximum(face_boxes, 0)
            #     for (x1, y1, x2, y2) in face_boxes:
            #         face_list.append((x1, y1, x2, y2))

            if rs.detections:
                for id, detection in enumerate(rs.detections):
                    bboxC = detection.location_data.relative_bounding_box
                    ih, iw, ic = frame.shape
                    bbox = int(bboxC.xmin * iw), int(bboxC.ymin * ih), \
                            int(bboxC.width * iw), int(bboxC.height * ih),
                    cv2.rectangle(frame, bbox, (255,0,255),2)
            

        cTime = time.time()
        fps_new = 1 / (cTime - pTime)
        pTime = cTime
        cv2.putText(frame, f'FPS: {int(fps_new)}', (20,70), cv2.FONT_HERSHEY_PLAIN, 3 , (0,255,0),2)
        
        cv2.imshow(WINDOW_NAME, frame)
        cv2.moveWindow(WINDOW_NAME, 0, 0)
        fps.update()

        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    fps.stop()

In [11]:
def open_window(width, height):
    cv2.namedWindow(WINDOW_NAME, cv2.WINDOW_NORMAL)
    cv2.resizeWindow(WINDOW_NAME, width, height)
    cv2.moveWindow(WINDOW_NAME, 0, 0)
    cv2.setWindowTitle(WINDOW_NAME, 'Camera Demo for Jetson Nano')

In [12]:
video_capture = open_cam_usb(1)
open_window(width, height)

start_app(video_capture)

video_capture.release()
cv2.destroyAllWindows()

/Users/dominhtuan/anaconda3/lib/python3.11/site-packages/google/protobuf/symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '


KeyboardInterrupt: 